# LLM 101 PoC, Tester

Goal: Test implementations interatively.

# Installs

In [1]:
# 

# Library

In [2]:
# import google.generativeai as genai
# import os

# from dotenv import load_dotenv
# from IPython.display import display
# from IPython.display import Markdown
# import textwrap
# from langchain_google_genai import ChatGoogleGenerativeAI
# from langchain import PromptTemplate
# from langchain.chains import LLMChain
# from langchain.chains import SimpleSequentialChain
# import requests
# from bs4 import BeautifulSoup
# import re
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain_google_genai import GoogleGenerativeAIEmbeddings
# from langchain_pinecone import PineconeVectorStore
# from pinecone import Pinecone, ServerlessSpec
# # from langchain.chains.question_answering import load_qa_chain
# # from langchain.prompts import ChatPromptTemplate
# from langchain.chains.combine_documents import create_stuff_documents_chain
# from langchain.chains import create_retrieval_chain
# from pprint import pprint
# from PyPDF2 import PdfReader

# Library

In [5]:
import sys  
sys.path.insert(1, '/home/jovyan/workspace/src')

In [28]:
import utils
import controller
import importlib
importlib.reload(utils)
importlib.reload(controller)

<module 'controller' from '/home/jovyan/workspace/src/controller.py'>

In [30]:
tool_utils = utils.UtilsLLM()

2024-05-21 10:30:34,244 - root - INFO - Utils initialized


In [31]:
tool_controller = controller.Controller('https://staffeng.com/guides/staff-archetypes/', 'what is the solver role?')

2024-05-21 10:30:34,736 - root - INFO - Utils initialized
2024-05-21 10:30:34,746 - root - INFO - Controller initialized


# Test

In [33]:
# Auth
tool_controller.authenticate()

2024-05-21 10:30:51,836 - root - INFO - Env vars loaded: True
2024-05-21 10:30:51,846 - root - INFO - GOOGLE_API_KEY len: 39
2024-05-21 10:30:51,853 - root - INFO - PINECONE_API_KEY len: 36
2024-05-21 10:30:51,857 - root - INFO - OPENAI_API_KEY len: 56
2024-05-21 10:30:51,890 - root - INFO - Succeed authenticating.


(True, 'Succeed authenticating.')

In [34]:
# Define embedding
tool_controller.define_embedding_model()

2024-05-21 10:30:53,342 - root - INFO - Succeed defining the embedding model
2024-05-21 10:30:53,351 - root - DEBUG - Embedding model defined: model='models/embedding-001' task_type=None google_api_key=None credentials=None client_options=None transport=None request_options=None


(GoogleGenerativeAIEmbeddings(model='models/embedding-001', task_type=None, google_api_key=None, credentials=None, client_options=None, transport=None, request_options=None),
 'Succeed defining the embedding model')

In [35]:
# --- Prepare content
# Check existence of Pineocone index
tool_controller.check_if_vector_store_index_already_exists_for_this_url('https://staffeng.com/guides/staff-archetypes/')

2024-05-21 10:30:54,868 - root - DEBUG - Checking URL https://staffeng.com/guides/staff-archetypes/ hashed to 2aa286980436145bc4d29577d69b9c69
2024-05-21 10:30:54,877 - root - DEBUG - Looking for 2aa286980436145bc4d29577d69b9c69
2024-05-21 10:30:55,195 - root - DEBUG - ... in ['2aa286980436145bc4d29577d69b9c69']
2024-05-21 10:30:55,382 - root - INFO - Specific index name exists in Pinecone
2024-05-21 10:30:55,388 - root - ERROR - A vector store index already exists for this URL: True


True

In [36]:
# Check url value
print(tool_controller.url)

https://staffeng.com/guides/staff-archetypes/


In [41]:
# Prepare vector store client
tool_controller.vector_store_loaded_client = tool_utils.set_vector_store_client_to_specific_index(tool_controller.url, tool_controller.embedding_model)

2024-05-21 10:32:22,669 - root - INFO - Succeed setting Pinecone client to a specific index


In [43]:
# Check client loaded vector store
print(tool_controller.vector_store_loaded_client)

In [44]:
# --- Prepare LLM chain
# Check if chain is ready
tool_controller._chain_is_prepared

False

In [45]:
# Define model choice
tool_controller.model_choice = "Gemini"

In [46]:
# Prepare chain
tool_controller.prepare_LLM_chain(tool_controller.vector_store_loaded_client)

2024-05-21 10:34:16,801 - root - INFO - LLM model defined as Gemini
2024-05-21 10:34:16,811 - root - INFO - Succeed defining the llm model
2024-05-21 10:34:16,819 - root - INFO - Succeed preparing prompt
2024-05-21 10:34:16,847 - root - INFO - Succeed building chain


In [48]:
# --- Ask question about the content
tool_controller.question = 'what is the solver role?'

In [49]:
# Ask to LLM
tool_controller.ask_question_to_llm(tool_controller.model_choice)

2024-05-21 10:35:53,471 - root - INFO - Succeed asking question about content using a retrieved documents chain
2024-05-21 10:35:53,475 - root - INFO - Question asked to Gemini
2024-05-21 10:35:53,482 - root - INFO - Q: what is the solver role?
2024-05-21 10:35:53,498 - root - INFO - A: The Solver digs deep into arbitrarily complex problems and finds an appropriate path forward. Some focus on a given area for long periods. Others bounce from hotspot to hotspot as guided by organizational leadership.
2024-05-21 10:35:53,509 - root - INFO - {'input': 'what is the solver role?', 'context': [Document(page_content="The Solver is most common in companies that think of individuals, rather than teams, as the atomic unit of planning and ownership.\nIn such companies, it's common to see the Solver become prevalent in the place of the Tech Lead.\nYou're less likely to encounter this role at traditionally managed sprint-centric companies until those companies become\nrelatively large or long-lived

(True,
 {'input': 'what is the solver role?',
  'context': [Document(page_content="The Solver is most common in companies that think of individuals, rather than teams, as the atomic unit of planning and ownership.\nIn such companies, it's common to see the Solver become prevalent in the place of the Tech Lead.\nYou're less likely to encounter this role at traditionally managed sprint-centric companies until those companies become\nrelatively large or long-lived enough to acquire their own varietal of technical debt.\nRight Hand"),
   Document(page_content="Influential architects dedicate their energy to maintaining an intimate understanding of the business' needs, their users' goals, and the relevant technical constraints.\nThey use that insight to identify and advocate for effective approaches within their area of focus,\nand do it with organizational authority that they've earned by demonstrating consistently good judgment.\nThe Architect role tends to evolve in relatively large comp

---
# End.